# Wind DT — Model Comparison Notebook

This notebook loads one or more CSV files produced by the pipeline's sinks (e.g., `preds_lgbm.csv`, `preds_svr.csv`), computes comparable metrics, and makes a few diagnostic plots.

**What you need:**
- CSVs with columns at least: `ts, y, y_hat` (and optionally `model, v, pi`).
- Put them in a folder and point `DATA_DIR` below.
- Set `RATED_POWER_KW` to your turbine's rated power for normalized errors.

> Charts use **matplotlib** (no seaborn), one chart per figure, and avoid explicit colors to match your environment rules.

In [1]:
# --- Configuration ---
from pathlib import Path
import pandas as pd
import numpy as np

# Path to the directory containing your CSV outputs (change as needed)
DATA_DIR = Path("out")  # e.g., Path(r"C:/Users/.../wind-dt/out")

# Glob pattern for CSVs to include. You can list specific files instead.
CSV_GLOB = "*.csv"  # e.g., "preds_*.csv"

# Turbine rated power (kW) for normalized error. Adjust to your turbine.
RATED_POWER_KW = 2050.0

# Optional: choose a time window for plots (None means use full range)
TIME_START = None  # e.g., "2020-06-01"
TIME_END   = None  # e.g., "2020-06-15"

# Downsample factor for plotting (plot every Nth point to keep figures light)
PLOT_EVERY = 10

In [3]:
# --- Load CSVs ---
import glob

files = sorted([Path(p) for p in glob.glob(str(DATA_DIR / CSV_GLOB))])
if not files:
    raise FileNotFoundError(f"No CSV files match pattern {CSV_GLOB} in {DATA_DIR.resolve()}")

dfs = []
for p in files:
    df = pd.read_csv(p)
    #if "ts" not in df.columns or "y" not in df.columns or "y_hat" not in df.columns:
        #raise ValueError(f"{p.name} must contain at least columns: ts, y, y_hat. Has: {list(df.columns)[:10]}")
    df["ts"] = pd.to_datetime(df["ts"], utc=True, errors="coerce")
    df = df.dropna(subset=["ts"])
    if "model" not in df.columns or df["model"].isna().all():
        stem = p.stem
        inferred = stem.replace("preds_", "")
        df["model"] = inferred
    for col in ["y","y_hat_H1","y_hat_D1","y_hat_W1","y_hat_M1","v","pi"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    if TIME_START is not None:
        df = df[df["ts"] >= pd.Timestamp(TIME_START, tz="UTC")]
    if TIME_END is not None:
        df = df[df["ts"] <= pd.Timestamp(TIME_END, tz="UTC")]
    dfs.append(df)

len(files), [f.name for f in files]

(1, ['preds_lag.csv'])

In [4]:
# --- Helper: infer sampling interval in hours per model ---
def infer_delta_hours(ts_series: pd.Series) -> float:
    if ts_series.size < 2:
        return np.nan
    dt = ts_series.sort_values().diff().dropna().median()
    return float(dt.total_seconds()) / 3600.0

sampling_by_model = {}
for df in dfs:
    model = df["model"].iloc[0]
    sampling_by_model[model] = infer_delta_hours(df["ts"])
sampling_by_model

{'lag_adapter': 0.16666666666666666}

In [ ]:
# --- Compute per-model metrics ---
def safe_mape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = np.abs(y_true) > 1e-6
    if mask.sum() == 0:
        return np.nan
    return float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0)

rows = []
# horizon columns we may encounter (including legacy single 'y_hat')
h_cols = ["y_hat", "y_hat_H1", "y_hat_D1", "y_hat_W1", "y_hat_M1"]
for df in dfs:
    model = df["model"].iloc[0]
    # find which horizon/prediction columns exist in this dataframe
    present_h = [c for c in h_cols if c in df.columns]
    if not present_h:
        # nothing to evaluate for this model
        continue
    for h in present_h:
        d = df.dropna(subset=["y", h]).copy()
        if d.empty:
            continue
        err = d[h] - d["y"]
        mae = float(np.mean(np.abs(err)))
        rmse = float(np.sqrt(np.mean(err**2)))
        nmae = mae / RATED_POWER_KW * 100.0
        dh = sampling_by_model.get(model, np.nan)
        energy_bias_kwh = float(np.nansum(err * (dh if np.isfinite(dh) else 0.0)))
        pi_mean = float(d["pi"].mean()) if "pi" in d.columns else np.nan
        pi_median = float(d["pi"].median()) if "pi" in d.columns else np.nan
        pi_p10 = float(d["pi"].quantile(0.1)) if "pi" in d.columns else np.nan
        pi_p90 = float(d["pi"].quantile(0.9)) if "pi" in d.columns else np.nan
        rows.append({
            "model": model,
            "horizon": (h.replace("y_hat_", "") if h.startswith("y_hat_") else ("H1" if h == "y_hat" else h)),
            "rows": int(len(d)),
            "sampling_h": float(dh) if np.isfinite(dh) else np.nan,
            "MAE_kW": mae,
            "RMSE_kW": rmse,
            "NMAE_%_of_rated": nmae,
            "MAPE_%": safe_mape(d["y"], d[h]),
            "EnergyBias_kWh": energy_bias_kwh,
            "PI_mean": pi_mean,
            "PI_median": pi_median,
            "PI_p10": pi_p10,
            "PI_p90": pi_p90
        })

metrics = pd.DataFrame(rows)
if not metrics.empty:
    metrics = metrics.sort_values(["NMAE_%_of_rated","RMSE_KW"]) if "RMSE_KW" in metrics.columns else metrics.sort_values(["NMAE_%_of_rated","RMSE_kW"])
metrics

KeyError: ['y_hat']

In [ ]:
# Save metrics table
metrics_path = DATA_DIR / "model_metrics.csv"
metrics.to_csv(metrics_path, index=False)
print(f"Saved metrics to: {metrics_path.resolve()}")
metrics

In [ ]:
# --- Merge multiple models for head-to-head plots ---
from functools import reduce

def prepare_for_merge(df):
    m = df.copy()
    mdl = m["model"].iloc[0]
    m = m[["ts","y","y_hat"]].rename(columns={"y_hat": f"y_hat_{mdl}"})
    return m

merged = reduce(lambda left,right: pd.merge(left, right, on=["ts","y"], how="outer"),
                [prepare_for_merge(df) for df in dfs])

merged = merged.sort_values("ts")
merged.head()

In [ ]:
# --- Plot: actual vs predicted (subset, top 1–3 models) ---
import matplotlib.pyplot as plt

pred_cols = [c for c in merged.columns if c.startswith("y_hat_")][:3]

fig = plt.figure(figsize=(12, 4))
if pred_cols:
    s = merged if PLOT_EVERY <= 1 else merged.iloc[::PLOT_EVERY, :]
    plt.plot(s["ts"], s["y"], label="Actual (y)")
    for c in pred_cols:
        plt.plot(s["ts"], s[c], label=c)
    plt.xlabel("Time")
    plt.ylabel("Power (kW)")
    plt.title("Actual vs Predicted (subset)")
    plt.legend()
else:
    plt.text(0.5, 0.5, "No prediction columns found", ha="center")
plt.show()

In [ ]:
# --- Scatter plots: y_hat vs y per model ---
for df in dfs:
    mdl = df["model"].iloc[0]
    d = df.dropna(subset=["y","y_hat"])
    if d.empty:
        continue
    fig = plt.figure(figsize=(4.5, 4.5))
    plt.scatter(d["y"], d["y_hat"], s=5)
    plt.xlabel("Actual y (kW)")
    plt.ylabel("Predicted y_hat (kW)")
    plt.title(f"y_hat vs y — {mdl}")
    plt.plot([d["y"].min(), d["y"].max()], [d["y"].min(), d["y"].max()])
    plt.show()

In [ ]:
# --- Residual histograms per model ---
for df in dfs:
    mdl = df["model"].iloc[0]
    d = df.dropna(subset=["y","y_hat"])
    if d.empty:
        continue
    err = (d["y_hat"] - d["y"]).values
    fig = plt.figure(figsize=(5, 3.5))
    plt.hist(err, bins=50)
    plt.xlabel("Residual (y_hat - y) kW")
    plt.ylabel("Count")
    plt.title(f"Residuals — {mdl}")
    plt.show()

# ⏱️ Multi‑Horizon Forecast Comparison (H1, D1, W1, M1)

*Appended automatically on 2025-10-08 19:44 UTC.*

This section compares **four different time‑horizon predictions** found in the CSV:

- `y_hat_H1` — short horizon (e.g., next step / hour)
- `y_hat_D1` — daily horizon
- `y_hat_W1` — weekly horizon
- `y_hat_M1` — monthly horizon

It computes common error metrics per **model × horizon** and visualizes them.


In [ ]:
# ==== Configuration ====
# Set the path to your CSV file here. If left as None, a small in-memory demo is used.
csv_path = None  # e.g., '/mnt/data/your_predictions.csv'

# Columns expected in the CSV:
# ts, model, v, y, y_hat_H1, y_hat_D1, y_hat_W1, y_hat_M1, pi


In [ ]:
# ==== Imports ====
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Dict, Tuple

# Display settings
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 140)


In [ ]:
# ==== Load Data ====
from io import StringIO

if csv_path is None:
    demo_csv = StringIO("""ts,model,v,y,y_hat_H1,y_hat_D1,y_hat_W1,y_hat_M1,pi
2020-02-14T12:00:00,lag_adapter,8.41154804229737,1312.77902832031,956.1858763868765,877.478747242494,1343.364147246193,473.18349406401177,
2020-02-14T12:10:00,lag_adapter,8.94811565876007,1563.40854492188,1211.7134553682765,661.0247264533829,1174.8725162788635,632.8199656171491,
""")
    df = pd.read_csv(demo_csv)
else:
    df = pd.read_csv(csv_path)

# Parse timestamps if present
if "ts" in df.columns:
    df["ts"] = pd.to_datetime(df["ts"], errors="coerce")

# Basic sanity check
required_cols = {"model","y","y_hat_H1","y_hat_D1","y_hat_W1","y_hat_M1"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns in CSV: {missing}")

df.head()


In [ ]:
# ==== Compute Metrics Per Model × Horizon ====

# Melt horizons into long format
h_cols = ["y_hat_H1", "y_hat_D1", "y_hat_W1", "y_hat_M1"]
long = df.melt(
    id_vars=[c for c in df.columns if c not in h_cols],
    value_vars=h_cols,
    var_name="horizon",
    value_name="y_hat",
)

# Normalize horizon labels like H1, D1, W1, M1
long["horizon"] = long["horizon"].str.replace("y_hat_", "", regex=False)

def safe_mape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    eps = 1e-9
    denom = np.where(np.abs(y_true) < eps, np.nan, np.abs(y_true))
    return np.nanmean(np.abs((y_true - y_pred) / denom)) * 100.0

def compute_metrics(group: pd.DataFrame) -> Dict[str, float]:
    y = group["y"].astype(float).to_numpy()
    yhat = group["y_hat"].astype(float).to_numpy()
    resid = y - yhat
    mae = float(np.mean(np.abs(resid)))
    rmse = float(np.sqrt(np.mean(resid**2)))
    mape = float(safe_mape(y, yhat))
    # R^2
    ss_res = float(np.sum(resid**2))
    ss_tot = float(np.sum((y - np.mean(y))**2)) if len(y) > 1 else np.nan
    r2 = 1.0 - ss_res/ss_tot if ss_tot and not math.isclose(ss_tot, 0.0) else np.nan
    return {"MAE": mae, "RMSE": rmse, "MAPE_%": mape, "R2": r2}

metrics = (
    long
    .dropna(subset=["y","y_hat"])
    .groupby(["model","horizon"], as_index=False)
    .apply(lambda g: pd.Series(compute_metrics(g)))
    .sort_values(["model","horizon"])
    .reset_index(drop=True)
)

metrics


In [ ]:
# ==== Display Metrics Table ====
from caas_jupyter_tools import display_dataframe_to_user
display_dataframe_to_user("Multi-Horizon Metrics by Model", metrics)
metrics


In [ ]:
# ==== Plot RMSE by Horizon (Grouped by Model) ====
# One figure: x=horizon, grouped bars by model

pivot_rmse = metrics.pivot(index="horizon", columns="model", values="RMSE")
ax = pivot_rmse.plot(kind="bar", figsize=(8,4))
ax.set_title("RMSE by Horizon and Model")
ax.set_xlabel("Horizon")
ax.set_ylabel("RMSE")
ax.legend(title="Model")
plt.tight_layout()
plt.show()


In [ ]:
# ==== Example Time Series Plots: Actual vs Predicted (Per Horizon) ====
# This produces four separate plots (one per horizon)
# If you have many points, consider slicing with a date range or tail().

for hz in ["H1","D1","W1","M1"]:
    col = f"y_hat_{hz}"
    if col not in df.columns:
        continue

    # Use one model at a time if multiple exist (or plot all predictions together if desired).
    # Here, we'll plot the first model's series to keep the plot uncluttered.
    first_model = df["model"].iloc[0]
    sub = df[df["model"] == first_model].copy()

    # If timestamps exist, sort by ts for a proper line chart
    if "ts" in sub.columns:
        sub = sub.sort_values("ts")

    plt.figure(figsize=(10,3))
    if "ts" in sub.columns and pd.api.types.is_datetime64_any_dtype(sub["ts"]):
        x = sub["ts"]
    else:
        x = np.arange(len(sub))

    plt.plot(x, sub["y"], label="y (actual)")
    plt.plot(x, sub[col], label=f"{col} (pred)")
    plt.title(f"Actual vs Predicted — {first_model} — {hz}")
    plt.xlabel("Time" if "ts" in sub.columns else "Index")
    plt.ylabel("Value")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
# ==== Residual Distribution by Horizon (All Models Combined) ====
import numpy as np

for hz in ["H1","D1","W1","M1"]:
    hz_long = long[long["horizon"] == hz].dropna(subset=["y","y_hat"])
    if hz_long.empty:
        continue
    resid = (hz_long["y"] - hz_long["y_hat"]).astype(float).to_numpy()
    plt.figure(figsize=(6,3))
    plt.hist(resid, bins=30)
    plt.title(f"Residuals Histogram — {hz}")
    plt.xlabel("Residual (y - y_hat)")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.show()


### Notes
- **MAPE** excludes points where `|y| < 1e-9` to avoid division-by-zero.
- If you want **per-horizon leaderboards**, sort `metrics` within each horizon by RMSE or MAE.
- To focus on a specific **date window**, filter `df` by `ts` before melting.
- If you have **prediction intervals** (`pi`), you can extend this section with coverage and sharpness metrics.
